# Deploying trained model which is saved on S3  

In [2]:
import boto3
import sagemaker
from sagemaker import get_execution_role

region = boto3.session.Session().region_name
role = get_execution_role()

In [3]:
import argparse
import os
import warnings

import pandas as pd
import pathlib
import numpy as np
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import models
from IPython import display

! pip install -U sagemaker


     |████████████████████████████████| 520 kB 23.1 MB/s            
  Preparing metadata (setup.py) ... done
  Using cached attrs-20.3.0-py2.py3-none-any.whl (49 kB)
  Created wheel for sagemaker: filename=sagemaker-2.82.2-py2.py3-none-any.whl size=719306 sha256=21b98c5d4d890286a9ce825fc6b4eef61cdf58ea42749862f69cc9dda2bf6f3c
  Stored in directory: /root/.cache/pip/wheels/72/a7/eb/4e1556b36181635cb1ad5e9ea4ff88b644ada1c96c93f899c5
Successfully built sagemaker
  Attempting uninstall: attrs
    Found existing installation: attrs 21.2.0
    Uninstalling attrs-21.2.0:
      Successfully uninstalled attrs-21.2.0
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.70.0
    Uninstalling sagemaker-2.70.0:
      Successfully uninstalled sagemaker-2.70.0
You should consider upgrading via the '/usr/local/bin/python3.8 -m pip install --upgrade pip' command.


# Model path and creating estimator 

In [4]:
from sagemaker.tensorflow.model import TensorFlowModel

model_path = "s3://sagemaker-studio-062044820001-7qbctb3w94p/Training/models/tensorflow-training-2022-03-30-15-25-44-677/output/model.tar.gz"
model = TensorFlowModel(model_data=model_path, role=role, framework_version="2.6")

# Model Deployment

In [5]:
instance_type = "ml.t2.medium"

predictor = model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
)

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-------!

In [6]:
features_loc = "pre-processed data/"

test_ds = tf.data.experimental.load(features_loc + "test/")

In [10]:
sample = []
labels = []
for spectrogram, label in test_ds.take(1):
      sample = spectrogram
      labels = label
        

sample = tf.expand_dims(sample,0)
result = predictor.predict(np.array(sample))
labels = np.array(labels)
labels

array(2)

In [11]:
result

{'predictions': [[1.37237716, -1.45104814]]}

In [8]:
np.argmax(np.array(result["predictions"][0]))

0